Problem statement: https://ovgu-ailab.github.io/idl2020w/ass9.html

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation,Dense, Flatten, MaxPooling2D, Dropout, Conv2D
from tensorflow.keras.metrics import categorical_crossentropy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load CIFAR Data set.
cifar10 = tf.keras.datasets.cifar10 # tf.keras.datasets.cifar10.load_data()
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

print(train_images.shape)
print(test_images.shape)
print(train_labels.shape)
print(train_images[0].shape)

input_shape = train_images[0].shape
#show the first image.
plt.imshow(train_images[7])

In [ ]:
data = tf.data.Dataset.from_tensor_slices(
    (train_images.reshape([-1, 32, 32, 3]).astype(np.float32) / 255, train_labels.reshape(-1,).astype(np.int32)))

test_data = tf.data.Dataset.from_tensor_slices(
    (test_images.reshape([-1, 32, 32, 3]).astype(np.float32) / 255, test_labels.reshape(-1,).astype(np.int32)))
num_classes = 10

In [ ]:
inputs = tf.keras.Input(32,32)
print(inputs)

In [ ]:
def get_model(units_in_layer,num_classes):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(32, 32,3)))

  for idx , units in enumerate(units_in_layer):
    model.add(tf.keras.layers.Dense(units=units))
  else:
    model.add(tf.keras.layers.Dense(units=10,activation='softmax'))

  return model

In [ ]:
def Genreate_CNN_Arch(conv_layers,dense_layers,num_classes,lastlayeractivation='softmax'):
  model = Sequential()

  for idx in range(len(conv_layers)):
    layer = conv_layers[idx]
    if idx == 0:
      print("Adding layer",idx)
      model.add(Conv2D(layer[0],(layer[1],layer[1]),strides=(layer[2],layer[2]),padding=layer[3],activation=layer[4],input_shape=(32,32,3)))
      model.add(MaxPooling2D(pool_size=(layer[5],layer[5]),strides=(layer[6],layer[6]),padding=layer[7]))
    else:
      print("Adding Next layer",idx)
      model.add(Conv2D(layer[0],(layer[1],layer[1]),strides=(layer[2],layer[2]),padding=layer[3],activation=layer[4]))
      model.add(MaxPooling2D(pool_size=(layer[5],layer[5]),strides=(layer[6],layer[6]),padding=layer[7]))

  model.add(Flatten())

  # Add Denselayer
  for idx in range(len(dense_layers)):
    layer = dense_layers[idx]
    model.add(Dense(units=layer[0],activation=layer[1]))

  #output layer.
  model.add(Dense(num_classes,activation=lastlayeractivation ))

  return model

In [ ]:
#@tf.function
def train_adversarial_models(model,epochs,data,epsilon,norm):
  for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    for step, (img_batch,lbl_batch) in enumerate(data):
      with tf.GradientTape(persistent=True) as tape:
        tape.watch(img_batch)
        logits = model(img_batch)
        xent = loss_fn(lbl_batch,logits)

      input_grads = tape.gradient(xent,img_batch)
      grads = tape.gradient(xent,model.trainable_variables)
      #print("grads: ",grads)
      optimizer.apply_gradients(zip(grads,model.trainable_variables))

      adveserial_img_batch = img_batch + (epsilon * normalize_input_grads(input_grads,norm))

      adveserial_img_batch = tf.stop_gradient(adveserial_img_batch)

      with tf.GradientTape() as tape:
        adv_logits = model(adveserial_img_batch)
        adv_xent = loss_fn(lbl_batch,adv_logits)
        
      adv_grads = tape.gradient(adv_xent,model.trainable_variables)
      #print("adv_grads: ",adv_grads)
      optimizer.apply_gradients(zip(adv_grads,model.trainable_variables))


      if not step % 100:
        print("original Loss : {} and Adversarial Loss: {}".format(xent,adv_xent))

def normalize_input_grads(inputs,type='sign'):
  if type == 'norm':
    inputs /= tf.norm(inputs)
  elif type == 'sign':
    inputs = tf.sign(inputs)

  return inputs


In [ ]:
def test_adv_Net(model,test_data,epsilon=0.01):
  
  for layer in model.layers:
    layer.trainable = False
  
  model.compile(optimizer=optimizer,loss=loss_fn,metrics='accuracy')
  total_loss = 0.0
  total_acc = 0.0
  count = 0
  for (img_batch,lbl_batch) in test_data:
    with tf.GradientTape(watch_accessed_variables=False) as tape:
      tape.watch(img_batch)
      logits = model(img_batch)
      xent = loss_fn(lbl_batch,logits)

    inupt_grads = tape.gradient(xent,img_batch)

    adv_test_data = img_batch + epsilon * normalize_input_grads(inupt_grads)
    test_loss, test_accuracy = model.evaluate(adv_test_data, lbl_batch)
    
    total_loss += test_loss

    total_acc += test_accuracy
    count += 1

  return total_loss/count, total_acc/count

In [68]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.optimizers.Adam(0.01)
# number_of_filters, filter_size, convstride, convpadding, activation, max_pool_size, maxpoolstrides, maxpoolpadding.
conv_layers = [[64,3,1,"valid",'relu',1,2,"valid"]]
#number_of_units, activation function.
dense_layers = [[128,'relu']]
norm = 'norm'
model_conv = Genreate_CNN_Arch(conv_layers,dense_layers,10)
model_conv.summary()

data = data.shuffle(50000).batch(128)
epochs = 50
epsilon = 0.01
train_adversarial_models(model_conv,epochs,data,epsilon,norm)


Adding layer 0
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 14400)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               1843328   
_________________________________________________________________
dense_17 (Dense)             (None, 10)                1290      
Total params: 1,846,410
Trainable params: 1,846,410
Non-trainable params: 0
_________________________________________________________________
Epoch: 0
original Loss : 2.2978782653808594 and Adversarial Loss: 9.568439483642578
original Lo

In [69]:
test_data_b = test_data.shuffle(10000).batch(128)
test_loss, test_acc = test_adv_Net(model_conv,test_data_b,epsilon)
print("test_loss: {} test_acc: {}".format(test_loss,test_acc))

1/1 [==============================] - 0s 99ms/step - loss: 40.8806 - accuracy: 0.3125
test_loss: 45.15069879459429 test_acc: 0.1473496835443038


In [70]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.optimizers.Adam(0.01)
# number_of_filters, filter_size, convstride, convpadding, activation, max_pool_size, maxpoolstrides, maxpoolpadding.
conv_layers = [[64,3,1,"valid",'relu',1,2,"valid"]]
#number_of_units, activation function.
dense_layers = [[128,'relu']]
norm = 'sign'
model_conv = Genreate_CNN_Arch(conv_layers,dense_layers,10)
model_conv.summary()

#data = data.shuffle(50000).batch(128)
epochs = 50
epsilon = 0.01
train_adversarial_models(model_conv,epochs,data,epsilon,norm)

Adding layer 0
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 14400)             0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               1843328   
_________________________________________________________________
dense_19 (Dense)             (None, 10)                1290      
Total params: 1,846,410
Trainable params: 1,846,410
Non-trainable params: 0
_________________________________________________________________
Epoch: 0
original Loss : 2.344310760498047 and Adversarial Loss: 20.94800567626953
original Los

In [71]:
test_data_b = test_data.shuffle(10000).batch(128)
test_loss, test_acc = test_adv_Net(model_conv,test_data_b,epsilon)
print("test_loss: {} test_acc: {}".format(test_loss,test_acc))

1/1 [==============================] - 0s 99ms/step - loss: 2.1935 - accuracy: 0.3750
test_loss: 2.4517706617524353 test_acc: 0.2837223101265823


In [72]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.optimizers.Adam(0.01)
# number_of_filters, filter_size, convstride, convpadding, activation, max_pool_size, maxpoolstrides, maxpoolpadding.
conv_layers = [[64,3,1,"valid",'relu',1,2,"valid"],[64,3,1,"valid",'relu',2,1,"valid"]]
#number_of_units, activation function.
dense_layers = [[128,'relu']]
norm = 'norm'
model_conv = Genreate_CNN_Arch(conv_layers,dense_layers,10)
model_conv.summary()

#data = data.shuffle(50000).batch(128)
epochs = 50
epsilon = 0.01
train_adversarial_models(model_conv,epochs,data,epsilon,norm)

Adding layer 0
Adding Next layer 1
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 9216)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 128)               1179776   
_________________________________________________________________
dense_21 (Dense)  

In [73]:
test_data_b = test_data.shuffle(10000).batch(128)
test_loss, test_acc = test_adv_Net(model_conv,test_data_b,epsilon)
print("test_loss: {} test_acc: {}".format(test_loss,test_acc))

1/1 [==============================] - 0s 113ms/step - loss: 4.2835 - accuracy: 0.1875
test_loss: 4.475504503974432 test_acc: 0.0981012658227848


In [75]:
seq_model = get_model([512,256,256,128,128],num_classes)
norm = 'norm'
train_adversarial_models(seq_model,epochs,data,epsilon,norm)

Epoch: 0
original Loss : 2.8244805335998535 and Adversarial Loss: 457.0640869140625
original Loss : 8564.529296875 and Adversarial Loss: 4852.7568359375
original Loss : 2002.291259765625 and Adversarial Loss: 2316.85986328125
original Loss : 614.5768432617188 and Adversarial Loss: 518.573486328125
Epoch: 1
original Loss : 802.4020385742188 and Adversarial Loss: 741.9290771484375
original Loss : 634.9983520507812 and Adversarial Loss: 550.6116333007812
original Loss : 607.681396484375 and Adversarial Loss: 515.923095703125
original Loss : 298.56170654296875 and Adversarial Loss: 221.6471710205078
Epoch: 2
original Loss : 208.97019958496094 and Adversarial Loss: 204.01318359375
original Loss : 347.4029846191406 and Adversarial Loss: 232.67127990722656
original Loss : 172.39254760742188 and Adversarial Loss: 184.875732421875
original Loss : 162.51226806640625 and Adversarial Loss: 190.42062377929688
Epoch: 3
original Loss : 177.50302124023438 and Adversarial Loss: 329.7176513671875
origin